# 卷積神經網路
CNN使用於影像辨識、聲音辨識等各種情況。在視覺辨識競賽中，深度學習使用的手法幾乎都是以CNN為基礎。本章將詳細說明CNN的結構，並且利用Python執行處理內容。
前面說明過的神經網路是，相鄰各層的所有神經元彼此相連，稱為**全連接( _fully connected_ )**，我們以Affine層的名稱，執行過全連接層。如圖所示，利用Affine層，可以建構出5層全連接的神經網路。<br>
![Cnn](./img/ch7-CNN.PNG)<br><br>

那用CNN會形成何種結構呢?如下圖所示<br>
![Cnn_full](./img/ch7.cnn_fully.PNG)<br>
CNN加入新的「Convolution層」與「polling層」。CNN各層的連接順序是「Convlution - ReLU - (Polling)」(有時會省略Polling層)。並於最後輸出層，使用「Affine - ReLU」組合，這是一般常見的CNN結構。


## 7.2 卷積層
在CNN中，出現了padding、stride等CNN專用的名詞。另外，傳遞在各層的資料變成具有形狀的資料(例如，三維資料)，與之前的全連接網路不同。因此，我們在這裡花多點時間，徹底介紹CNN所使用的卷積層結構。


### 7.2.1 全連接層的問題
前面介紹的全連接神經網路，使用了全連接層( Affine層 )。全連接層連接相鄰各層的所有神經元，可以決定任意輸出數量。<br><br>

全連接層的問題在於，他會「忽略」資料的形狀。例如，輸入資料為影像，影像通常是有水平、垂直、色板方向的三維形狀。可是，輸入全連接層時，三維資料必須變成平面(一維)。以前面提過的MNIST資料集為例子，輸入影像就是(1,28,28)，1色板、垂直28像素、水平28像素的形狀，全連接層卻會把這個形狀變成一行784(28x28)個資料，做為輸入資料，輸入最初的Affine層。<br><br>

然而，卷積層(Convolution層)能維持形狀。如果是影像，輸入資料可以當作三維資料來處理，對上下一層輸出同樣是三維的資料。因此，CNN(可能)可以正確了解影像等含有形狀的資料。<br><br>

有時，我們會把CNN的卷積層輸出入的資料稱作**輸入特徵圖(_input feature map_)**，與**輸出特徵圖(_output feature map_)**，而本書將上面兩個專有名詞與「特徵圖」視為等意詞。


### 7.2.2 卷積運算
![卷積運算步驟](./img/7.2.2.PNG)<br>
卷積運算如圖所示，針對輸入資料，以固定的間隔，一邊移動，一邊套用濾鏡視窗。在各個位置乘上濾鏡的元素與對應輸入的元素，並計算總合(這個部分也稱為稽和運算)。再將結果儲存在對應輸出的位置。在全部的位置執行這格過程，可以得到積運算的輸出。<br><br>

在全連接的神經網路中，除了權重參數之外，還有偏權值。CNN的濾鏡參數對應的是前面提到的「權重」而且在CNN中，也有偏權值，下圖表式卷積運算套用濾鏡後的運算。包含偏權值的卷積運算處理流程。<br>
![7.2.2卷積運算的偏權值](./img/7.2.2.1.PNG)<br>
可以看到，偏權值隨時都只有一值，這個值要加在套用濾鏡後的所有元素。


## 7.2.3 填補(padding)
進行卷積處理之前，必須在輸入資料的周圍填上固定的資料(例如0)，這個動作稱作**填補( _padding_ )**，也是在卷積運算中，常用的處理。如圖所示，圖中對於大小為(4x4)的輸入資料，進行寬度為1的填補。<br>
![7.2.3padding](./img/7.2.3.PNG)<br><br>

{Note}<br>
使用填補的理由是為了調整輸出大小。假設在大小為(4x4)的輸入資料中，套用(3x3)的濾鏡時，輸出大小變成(2x2)，輸出資料只會縮小輸入資料的2個元素。如此一來，在反覆進行卷積運算的多層網路中，就會造成問題。如果每次進行卷積運算時，空間就會縮小的話，到了某個階段，輸出大小就會變成1，而無法再進行卷積運算，使用填補就是為了避免出現這樣的狀況。將填補大小設定為1，相對於輸入大小為(4x4)，輸出大小也會保持(4x4)。因此，藉由卷積運算，可以維持固定的空間大小，將資料傳遞給下一層。


### 7.2.4 步伐
套用濾鏡的位置間隔稱為**步伐( _stride_ )**，若我們把步伐變大，輸出大小就會變小，但我們把填補變大，輸出大小就會變大；所以，我們要把這種關係公式化。<br><br>

這裡假設輸入大小為$(H,W)$，濾鏡大小為$(FH,FW)$，輸出大小為$(OH,OW)$，填補為$P$，步伐為$S$，可以利用以下算式，計算輸出大小。<br>
$
OH = \frac{H+2P-FH}{S} + 1 \tag{7.1}
$

$
OW = \frac{W+2P-FW}{S} + 1 \tag{7.1}
$<br>
我們來練習一題：<br>
* 輸入大小：(28, 31)、填補：2、步伐：3、濾鏡大小：(5, 5)<br>

$
OH = \frac{28+(2*2)-5}{3} + 1 = 10
$<br>

$
OW = \frac{31+(2*2)-5}{3} + 1 = 11
$<br><br>

這裡必須注意的是，我們一定要讓算式(7.1)整除，假如輸出大小無法整除(結果為小數)，那麼我們就必須採取輸出錯誤的對策。


### 7.2.5 三維資料的卷積運算
三維資料包含，水平、垂直方向的形狀資料，以及色板方向所形成的三維資料。圖為三維資料的卷積運算的步驟。<br>
![三維資料的卷積運算範例](./img/7.2.5.PNG)<br>
這裡要注意的是，濾鏡大小可以隨意設定，但是每個色板的濾鏡大小要一致。這個範例的濾鏡大小是(3x3)，因此色板數量只能設定成3。


### 7.2.6 用區塊來思考
我們也可以將三維卷積運算中的資料或濾鏡想像成立體區塊，比較容易思考，例如，色板數C、高度H、寬度W的資料形狀會寫成(C, H, W )，濾鏡則由色板數C、濾鏡高度FH、寬度FW來表示(C, FH, FW)。<br>
![多個濾鏡的卷積運算](./img/7.2.6.jpg)<br>
由圖可知，套用FN個濾鏡，也會產生FN個輸出的特徵圖。<br><br>

在卷積運算中(和全連接層相同)，也有偏權值與權重，偏權值是每個色板只有一個資料。這裡偏權值形狀是(FN, 1, 1)，濾鏡輸出結果的形狀是(FN, OH, OW)。<br>
![卷積運算的處理流程(加上偏權值)](./img/7.2.6.1.jpg)


## 7.3 池化層
池化層是縮小垂直、水平空間的運算，如圖所示：<br>
![池化層運算](./img/7.3.PNG)<br>
其中，「最大池化」是用來取得每個視窗的最大值，而「平均池化」則是指，從每個視窗取出平均值。


### 7.3.1 池化層的特色
池化層有以下特色<br>
* 沒有學習參數<br>
&emsp; 池化層與卷積層不同，沒有學習參數。池化層指進行從目標區域取得最大值( 或平均值 )的處理，所以沒有必需學習的參數存在。<br>
* 色板數量不變<br>
&emsp; 輸入資料與輸出資料的色板數量不會隨著池化算而改變，各個色板進行獨立的運算。<br>
* 對微小位置變化很穩健<br>
&emsp; 即使輸入資料出現小偏差，池化仍會回傳相同結果(因為只取最大或平均池化)。因此，對輸入資料的微小偏差很穩健。


## 7.4 執行卷積層 / 池化層
或許我們直覺認為執行卷積層與池化層很複雜，但其實只要使用某個「妙招」，就可以輕鬆完成，而在執行的類別中，含有forwawrd與backward等方法。

